In [39]:
import numpy as np
import cv2
import winsound

In [40]:
looking_centre=np.load('looking_centre.npy')
looking_right=np.load('looking_right.npy')
        

In [41]:
looking_centre=looking_centre.reshape(300,50*50*3)
looking_right=looking_right.reshape(300,50*50*3)


In [42]:
looking_centre.shape


(300, 7500)

In [43]:
X=np.r_[looking_centre,looking_right]

In [44]:
label=np.zeros(X.shape[0])

In [45]:
label[300:]=1.0

In [46]:
names={0:'SAFE',1:'CHEATING'}

In [47]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [48]:
x_train,x_test,y_train,y_test=train_test_split(X,label,test_size=0.20)

In [49]:
x_train.shape

(480, 7500)

In [50]:
from sklearn.decomposition import PCA

In [51]:
pca=PCA(n_components=3)
x_train=pca.fit_transform(x_train)

In [52]:
x_train[0]

array([-2547.89827192,   -31.06206099,  -793.50634084])

In [53]:
x_train.shape


(480, 3)

In [54]:
x_train,x_test,y_train,y_test=train_test_split(X,label,test_size=0.11)

In [55]:
svm=SVC()
svm.fit(x_train,y_train)

SVC()

In [56]:

y_pred=svm.predict(x_test)

In [57]:
haar_data = cv2.CascadeClassifier('data.xml')

In [58]:
capture = cv2.VideoCapture(0)
data=[]
font=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
while True:
    flag,img=capture.read()
    if flag:
        faces=haar_data.detectMultiScale(img)
        n_faces = len(faces)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(200,162,255),4)
            face=img[y:y+h,x:x+h,:]
            face=cv2.resize(face,(50,50))
            face=face.reshape(1,-1)
            pred=svm.predict(face)[0]
            n=names[int(pred)]
            if n_faces>1:
                cv2.putText(img,"Cheating",(x,y),font,1,(222,234,222),2)
                cv2.imshow('result',img)
            else:
                cv2.putText(img,"Safe",(x,y),font,1,(222,234,222),2)
                cv2.imshow('result',img)
                
        if cv2.waitKey(2)==27:
            break
capture.release()
cv2.destroyAllWindows()